In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')


import numpy as np
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
from sys import argv
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time

libc = CDLL(find_library('c'))
cstdout = c_void_p.in_dll(libc, 'stdout')

In [2]:
class layers:
    def __init__(self):
        self.layertypes = []
        self.weights = []
        self.biases = []
        self.numlayer = 0
        self.ffn_counter = 0

def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    #return v.reshape((v.size,1))
    return v

def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    return v.reshape((v.size,1))
    #return v

def balanced_split(text):
    i = 0
    bal = 0
    start = 0
    result = []
    while i < len(text):
        if text[i] == '[':
            bal += 1
        elif text[i] == ']':
            bal -= 1
        elif text[i] == ',' and bal == 0:
            result.append(text[start:i])
            start = i+1
        i += 1
    if start < i:
        result.append(text[start:i])
    return result

def parse_matrix(text):
    i = 0
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])

def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]
    i = 0
    res = layers()
    while i < len(lines):
        if lines[i] in ['ReLU', 'Affine']:
            W = parse_matrix(lines[i+1])
            b = parse_bias(lines[i+2])
            res.layertypes.append(lines[i])
            res.weights.append(W)
            res.biases.append(b)
            res.numlayer+= 1
            i += 3
        else:
            raise Exception('parse error: '+lines[i])
    return res
   
def parse_spec(text):
    text = text.replace("[", "")
    text = text.replace("]", "")
    with open('dummy', 'w') as my_file:
        my_file.write(text)
    data = np.genfromtxt('dummy', delimiter=',',dtype=np.double)
    low = np.copy(data[:,0])
    high = np.copy(data[:,1])
    return low,high

def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]
    num_pixels = len(image)
    LB_N0 = image - epsilon
    UB_N0 = image + epsilon
     
    for i in range(num_pixels):
        if(LB_N0[i] < 0):
            LB_N0[i] = 0
        if(UB_N0[i] > 1):
            UB_N0[i] = 1
    return LB_N0, UB_N0


def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)
    cst = pointer(linexpr0.contents.cst)
    elina_scalar_set_double(cst.contents.val.scalar, bias)
    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0,i,weights[i])
    return linexpr0

In [3]:
from abc import abstractmethod
class ElinaToolbox():
    @abstractmethod
    def hypercubeToNumpy(man, HC, nb_elts):
        """Return inf and sup bounds"""
        bounds = elina_abstract0_to_box(man, HC)
        return (
            [elt.contents.inf.contents.val.dbl for elt in bounds[:nb_elts]], 
            [elt.contents.sup.contents.val.dbl for elt in bounds[:nb_elts]]
        )
    @abstractmethod
    def NumpyToHypercube(man, LB, UB):
        num_pixels = len(LB)
        itv = elina_interval_array_alloc(num_pixels)
        for i in range(num_pixels):
            elina_interval_set_double(itv[i], LB[i], UB[i])
        HC = elina_abstract0_of_box(man, 0, num_pixels, itv)
        elina_interval_array_free(itv,num_pixels)
        return HC

class SolverLayer():
    SUPPORTED_LT = ['ReLU', 'Affine']
    def __init__(self, layertypes, weights, biases):
        for layertype in layertypes:
            assert layertype in self.SUPPORTED_LT, 'Net type not supported'
        self.layertypes=layertypes
        self.weights=weights
        self.biases=biases

In [4]:
class ElinaLayers(SolverLayer):
    def __init__(self, layertypes, weights, biases, man):
        super().__init__(layertypes, weights, biases)
        self.total_layers=len(weights)
        self.man = man
        self.HC = None
        
        
    def __del__(self):
        elina_abstract0_free(self.man, self.HC)
        elina_manager_free(self.man)
    
    def compute(self, LB, UB, input_layerno=None, output_layerno=None):
        """
            Return the output values range for i=input_layerno + 1 to i = output_layerno + 1
        """
        self.HC = ElinaToolbox.NumpyToHypercube(self.man, LB, UB)
        lower_bounds = [0. for _ in range(self.total_layers)]
        upper_bounds = [0. for _ in range(self.total_layers)]
        for layerno in range(self.total_layers):
            layerdim = len(self.biases[layerno])
            self.propagate_hypercube(layerno)
            LB_layerno, UB_layerno = ElinaToolbox.hypercubeToNumpy(self.man, self.HC, layerdim)
            lower_bounds[layerno], upper_bounds[layerno]  = LB_layerno, UB_layerno

        return lower_bounds, upper_bounds
        
    def propagate_hypercube(self, layer_no):
        weights, biases = self.weights[layer_no], self.biases[layer_no]
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        
        dims = elina_abstract0_dimension(self.man, self.HC)
        
        self.num_in_pixels = dims.intdim + dims.realdim
        self.num_out_pixels = len(weights)
        
        self.add_output_dim()
        self.compute_affine(weights, biases)
        self.remove_input_dim()
        if(self.layertypes[layer_no]=='ReLU'):
            self.compute_relu()
        
    def add_output_dim(self):
        dimadd = elina_dimchange_alloc(0,self.num_out_pixels) 
        for i in range(self.num_out_pixels): 
            dimadd.contents.dim[i] = self.num_in_pixels
        elina_abstract0_add_dimensions(self.man, True, self.HC, dimadd, False)
        elina_dimchange_free(dimadd)
        
    def remove_input_dim(self):
        dimrem = elina_dimchange_alloc(0,self.num_in_pixels)
        for i in range(self.num_in_pixels): 
            dimrem.contents.dim[i] = i
        elina_abstract0_remove_dimensions(self.man, True, self.HC, dimrem)
        elina_dimchange_free(dimrem)
        
    def compute_affine(self, weights, biases):
        for i in range(self.num_out_pixels):
            self.HC = elina_abstract0_assign_linexpr_array(
                man=self.man, 
                destructive=True, 
                org=self.HC, 
                tdim=ElinaDim(self.num_in_pixels + i), 
                linexpr_array=generate_linexpr0(weights[i],biases[i],self.num_in_pixels), 
                size=1, 
                dest=None
            )
            
    def compute_relu(self):
        self.HC = relu_box_layerwise(
            man=self.man,
            destructive=True,
            elem=self.HC,
            start_offset=0, 
            num_dim=self.num_out_pixels
        )

In [5]:
class GurobiLayers(SolverLayer):  
    def __init__(self, layertypes, weights, biases, lower_bounds, upper_bounds):
        super().__init__(layertypes, weights, biases)
        self.m = Model(name='NN')
        self.m.setParam('OutputFlag', False)
        self.dim = [len(elt) for _, elt in enumerate(biases)]
        self.reludim = self.dim[:] if layertypes[-1] == "ReLU" else self.dim[:-1]
        self.lower_bounds, self.upper_bounds = lower_bounds, upper_bounds
        
        self.define_variables()
        self.add_relu_constraints()
        self.add_hidden_layer_constraints()
    
    def define_variables(self):
        self.his = [[(
                self.m.addVar(lb=self.lower_bounds[i][j],ub=self.upper_bounds[i][j],vtype=GRB.CONTINUOUS, name=f"Hidden_{i}_{j}")
                if i == 0 else 
                self.m.addVar(lb=-np.inf, vtype=GRB.CONTINUOUS, name=f"Hidden_{i}_{j}"))
            for j in range(self.dim[i])] 
            for i in range(len(self.dim))]
        
        self.ris = [[self.m.addVar(lb=-np.inf, vtype=GRB.CONTINUOUS, name=f"ReLU_{i}_{j}") 
            for j in range(self.dim[i])] 
            for i in range(len(self.reludim))]
            
        self.m.update()

    def add_relu_constraints(self):
        for i in range(len(self.reludim)):
            for j in range(self.reludim[i]):
                inf, sup = self.lower_bounds[i][j], self.upper_bounds[i][j]

                if (inf >= 0):
                    self.m.addConstr(self.ris[i][j] == self.his[i][j])
                elif (sup <= 0):
                    self.m.addConstr(self.ris[i][j] == 0)
                else:
                    k, t = sup / (sup - inf), -sup * inf / (sup - inf)

                    self.m.addConstr(self.ris[i][j] >= 0)
                    self.m.addConstr(self.ris[i][j] >= self.his[i][j])
                    self.m.addConstr(self.ris[i][j] <= k * self.his[i][j] + t)

        self.m.update()
    
    def add_hidden_layer_constraints(self):
        for i in range(1, len(self.dim)):
            for j in range(self.dim[i]):
                self.m.addConstr(
                    self.his[i][j] == LinExpr(self.weights[i][j, :], self.ris[i-1]) + self.biases[i][j]
                )

        self.m.update()
    
    def optimize_one(self, j, minimize=True, write=False):
        self.m.reset()
        self.m.setObjective(self.his[-1][j], GRB.MINIMIZE  if minimize else GRB.MAXIMIZE)
        if write: self.m.write("models/model_j={j}_min={minimize}.lp")
        self.m.optimize()
        try:
            return self.m.objVal
        except:
            print("Can't find {bound} bound for neuron {j}".format(bound="lower" if minimize else "upper", j=j))
            return None
                           
    def compute(self, write=False):
        return (
            [self.optimize_one(j=j, minimize=True, write=write) for j in range(self.dim[-1])],
            [self.optimize_one(j=j, minimize=False, write=write) for j in range(self.dim[-1])],
        )

In [6]:
def open_files(netname, specname, epsilon):
    with open(netname, 'r') as netfile:
        netstring = netfile.read()
    with open(specname, 'r') as specfile:
        specstring = specfile.read()
        
    nn = parse_net(netstring)
    x0_low, x0_high = parse_spec(specstring)
    LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
    
    return nn, LB_N0, UB_N0

In [7]:
netname = "../mnist_nets/mnist_relu_3_10.txt"
specname = "../mnist_images/img10.txt"
epsilon = 0.01

nn, LB_N0, UB_N0 = open_files(netname, specname, epsilon)
layertypes, weights, biases = nn.layertypes, nn.weights, nn.biases
man = elina_box_manager_alloc()

el = ElinaLayers(layertypes, weights, biases, man)
lb, ub = el.compute(LB_N0, UB_N0)
print(f"Elina lower bound = {lb[-1]}")
print(f"Elina upper bound = {ub[-1]}")
del el

print("")

gl = GurobiLayers(layertypes, weights, biases, lb, ub)
out_lb, out_ub = gl.compute()

print(f"Lower bound = {out_lb}")
print(f"Upper bound = {out_ub}")

Elina lower bound = [9.235913312139143, -0.0, 1.1542118512498287, -0.0, -0.0, 5.111503056406706, -0.0, -0.0, -0.0, -0.0]
Elina upper bound = [13.222227617213552, 0.0, 3.0001427711185724, 0.9395664532817398, 0.0, 7.097974193980767, 2.5864704001602146, 0.0, 2.7111386759659646, 0.9824644770588353]

Academic license - for non-commercial use only
Lower bound = [20.123494522669688, -15.44672815242845, -0.1677645086874502, -4.0663780158617175, -9.345297883578702, 10.051955313251693, 7.263421618718439, -9.434450922168187, -9.056717732677187, -6.095986170046755]
Upper bound = [26.668529787009636, -11.942062717829087, 1.8028611595848947, -0.43456060821271913, -7.289842820760105, 14.282086917165241, 10.984688223999939, -4.664657429808076, -3.7114456790302834, -3.7641972534998867]
